In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
spark = SparkSession.builder.appName('dfuc').getOrCreate()

In [0]:
dfuc = spark.read.option('header','true').csv('dbfs:/FileStore/shared_uploads/martinsmv.laurano@gmail.com/cars.csv')

In [0]:
dfuc = dfuc.drop('engine_type').drop('has_warranty').drop('state').drop('drivetrain').drop('is_exchangeable').drop('location_region').drop('number_of_photos').drop('up_counter').drop('feature_0').drop('feature_1').drop('feature_2').drop('feature_3').drop('feature_4').drop('feature_5').drop('feature_6').drop('feature_7').drop('feature_8').drop('feature_9')

In [0]:
dfuc.show(5)

+---------+--------+-----------+--------+------------+----+-----------+-----+---------+-------------------------+---+----------+
|Montadora|  Modelo|Transmissão|     Cor|Kilometragem| Ano|Combustível|Motor|Categoria|Duração do Anúncio (dias)|GNV|Valor (R$)|
+---------+--------+-----------+--------+------------+----+-----------+-----+---------+-------------------------+---+----------+
|   Subaru| Outback| Automático|   Prata|      190000|2010|   Gasolina|  2.5|      SUV|                       16|Não|   59950.0|
|   Subaru| Outback| Automático|    Azul|      290000|2002|   Gasolina|  3.0|      SUV|                       83|Não|   27500.0|
|   Subaru|Forester| Automático|Vermelho|      402000|2001|   Gasolina|  2.5|      SUV|                      151|Não|   15400.0|
|   Subaru| Impreza|     Manual|    Azul|       10000|1999|   Gasolina|  3.0|    Sedan|                       86|Não|   54995.0|
|   Subaru|  Legacy| Automático|   Preto|      280000|2001|   Gasolina|  2.5|      SUV|          

In [0]:
dfuc = dfuc.withColumnRenamed('manufacturer_name','Montadora').withColumnRenamed('model_name','Modelo').withColumnRenamed('transmission','Transmissão').withColumnRenamed('color','Cor').withColumnRenamed('odometer_value','Kilometragem').withColumnRenamed('year_produced','Ano').withColumnRenamed('engine_fuel','Combustível').withColumnRenamed('engine_has_gas','Tem GNV').withColumnRenamed('engine_capacity','Motor').withColumnRenamed('body_type','Categoria').withColumnRenamed('price_usd','Preço (US$)').withColumnRenamed('duration_listed','Duração do Anúncio (dias)')

In [0]:
dfuc.show(5)

+---------+--------+-----------+--------+------------+----+-----------+-----+---------+-------------------------+---+----------+
|Montadora|  Modelo|Transmissão|     Cor|Kilometragem| Ano|Combustível|Motor|Categoria|Duração do Anúncio (dias)|GNV|Valor (R$)|
+---------+--------+-----------+--------+------------+----+-----------+-----+---------+-------------------------+---+----------+
|   Subaru| Outback| Automático|   Prata|      190000|2010|   Gasolina|  2.5|      SUV|                       16|Não|   59950.0|
|   Subaru| Outback| Automático|    Azul|      290000|2002|   Gasolina|  3.0|      SUV|                       83|Não|   27500.0|
|   Subaru|Forester| Automático|Vermelho|      402000|2001|   Gasolina|  2.5|      SUV|                      151|Não|   15400.0|
|   Subaru| Impreza|     Manual|    Azul|       10000|1999|   Gasolina|  3.0|    Sedan|                       86|Não|   54995.0|
|   Subaru|  Legacy| Automático|   Preto|      280000|2001|   Gasolina|  2.5|      SUV|          

In [0]:
dfuc = dfuc.withColumn('Preço (US$)',dfuc['Preço (US$)'].cast('float'))

In [0]:
dfuc.printSchema()

root
 |-- Montadora: string (nullable = true)
 |-- Modelo: string (nullable = true)
 |-- Transmissão: string (nullable = true)
 |-- Cor: string (nullable = true)
 |-- Kilometragem: string (nullable = true)
 |-- Ano: string (nullable = true)
 |-- Combustível: string (nullable = true)
 |-- Tem GNV: string (nullable = true)
 |-- Motor: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Preço (US$): float (nullable = true)
 |-- Duração do Anúncio (dias): string (nullable = true)



In [0]:
dfuc = dfuc.dropna(how='any')

In [0]:
dfuc = dfuc.withColumn('GNV',when(dfuc['Tem GNV'] == 'True','Sim').when(dfuc['Tem GNV'] == 'False','Não').otherwise(dfuc['Tem GNV']))

In [0]:
dfuc.dropDuplicates(['GNV']).show()

+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
|Montadora| Modelo|Transmissão|   Cor|Kilometragem| Ano|Combustível|Tem GNV|Motor|Categoria|Preço (US$)|Duração do Anúncio (dias)|GNV|
+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
|   Subaru|Outback|  automatic|silver|      190000|2010|   gasoline|  False|  2.5|universal|    10900.0|                       16|Não|
|   Subaru| Legacy|  automatic| black|      299000|1994|        gas|   True|  2.2|universal|      650.0|                       85|Sim|
+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+



In [0]:
dfuc.show(1)

+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
|Montadora| Modelo|Transmissão|   Cor|Kilometragem| Ano|Combustível|Tem GNV|Motor|Categoria|Preço (US$)|Duração do Anúncio (dias)|GNV|
+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
|   Subaru|Outback|  automatic|silver|      190000|2010|   gasoline|  False|  2.5|universal|    10900.0|                       16|Não|
+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
only showing top 1 row



In [0]:
dfuc.dropDuplicates(['Transmissão']).show()

+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
|Montadora| Modelo|Transmissão|   Cor|Kilometragem| Ano|Combustível|Tem GNV|Motor|Categoria|Preço (US$)|Duração do Anúncio (dias)|GNV|
+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
|   Subaru|Outback|  automatic|silver|      190000|2010|   gasoline|  False|  2.5|universal|    10900.0|                       16|Não|
|   Subaru|Impreza| mechanical|  blue|       10000|1999|   gasoline|  False|  3.0|    sedan|     9999.0|                       86|Não|
+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+



In [0]:
dfuc = dfuc.withColumn('Transmissão',when(dfuc['Transmissão'] == 'automatic','Automático').when(dfuc['Transmissão'] == 'mechanical','Manual').otherwise(dfuc['Transmissão']))

In [0]:
dfuc.dropDuplicates(['Transmissão']).show()

+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
|Montadora| Modelo|Transmissão|   Cor|Kilometragem| Ano|Combustível|Tem GNV|Motor|Categoria|Preço (US$)|Duração do Anúncio (dias)|GNV|
+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+
|   Subaru|Outback| Automático|silver|      190000|2010|   gasoline|  False|  2.5|universal|    10900.0|                       16|Não|
|   Subaru|Impreza|     Manual|  blue|       10000|1999|   gasoline|  False|  3.0|    sedan|     9999.0|                       86|Não|
+---------+-------+-----------+------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+



In [0]:
cor = dfuc.dropDuplicates(['Cor'])
cor.select('Cor').show()

+------+
|   Cor|
+------+
|violet|
|orange|
|  grey|
| green|
|yellow|
|silver|
| other|
| white|
|   red|
| black|
| brown|
|  blue|
+------+



In [0]:
dfuc = dfuc.withColumn('Cor',when(dfuc['Cor'] == 'violet','Roxo').when(dfuc['Cor'] == 'orange','Laranja').when(dfuc['Cor'] == 'grey','Cinza').otherwise(dfuc['Cor']))
dfuc = dfuc.withColumn('Cor',when(dfuc['Cor'] == 'green','Verde').when(dfuc['Cor'] == 'yellow','Amarelo').when(dfuc['Cor'] == 'silver','Prata').otherwise(dfuc['Cor']))
dfuc = dfuc.withColumn('Cor',when(dfuc['Cor'] == 'other','Outro').when(dfuc['Cor'] == 'white','Branco').when(dfuc['Cor'] == 'red','Vermelho').otherwise(dfuc['Cor']))
dfuc = dfuc.withColumn('Cor',when(dfuc['Cor'] == 'black','Preto').when(dfuc['Cor'] == 'brown','Marrom').when(dfuc['Cor'] == 'blue','Azul').otherwise(dfuc['Cor']))

In [0]:
cor_new = dfuc.dropDuplicates(['Cor'])
cor_new.select('Cor').show()

+--------+
|     Cor|
+--------+
|   Preto|
|  Marrom|
| Laranja|
|    Azul|
|   Prata|
|    Roxo|
|  Branco|
|   Verde|
|Vermelho|
|   Cinza|
| Amarelo|
|   Outro|
+--------+



In [0]:
combustivel = dfuc.dropDuplicates(['Combustível'])
combustivel.select('Combustível').show()

+-------------+
|  Combustível|
+-------------+
|     gasoline|
|          gas|
|hybrid-diesel|
|       diesel|
|hybrid-petrol|
+-------------+



In [0]:
dfuc = dfuc.withColumn('Combustível',when(dfuc['Combustível'] == 'gasoline','Gasolina').when(dfuc['Combustível'] == 'gas','Gás').otherwise(dfuc['Combustível']))
dfuc = dfuc.withColumn('Combustível',when(dfuc['Combustível'] == 'hybrid-diesel','Híbrido').when(dfuc['Combustível'] == 'diesel','Diesel').otherwise(dfuc['Combustível']))
dfuc = dfuc.withColumn('Combustível',when(dfuc['Combustível'] == 'hybrid-petrol','Híbrido').otherwise(dfuc['Combustível']))

In [0]:
combustivel_new = dfuc.dropDuplicates(['Combustível'])
combustivel_new.select('Combustível').show()

+-----------+
|Combustível|
+-----------+
|     Diesel|
|    Híbrido|
|   Gasolina|
|        Gás|
+-----------+



In [0]:
categoria = dfuc.dropDuplicates(['Categoria'])
categoria.select('Categoria').show()

+---------+
|Categoria|
+---------+
|      van|
|  minivan|
|cabriolet|
|    coupe|
|limousine|
|      suv|
|    sedan|
|  minibus|
|hatchback|
| liftback|
|   pickup|
|universal|
+---------+



In [0]:
dfuc = dfuc.withColumn('Categoria',when(dfuc['Categoria'] == 'van','Van').when(dfuc['Categoria'] == 'minivan','Minivan').when(dfuc['Categoria'] == 'cabriolet','Conversível').otherwise(dfuc['Categoria']))
dfuc = dfuc.withColumn('Categoria',when(dfuc['Categoria'] == 'coupe','Coupê').when(dfuc['Categoria'] == 'limousine','Limosine').when(dfuc['Categoria'] == 'suv','SUV').otherwise(dfuc['Categoria']))
dfuc = dfuc.withColumn('Categoria',when(dfuc['Categoria'] == 'sedan','Sedan').when(dfuc['Categoria'] == 'minibus','Minivan').when(dfuc['Categoria'] == 'hatchback','Hatch').otherwise(dfuc['Categoria']))
dfuc = dfuc.withColumn('Categoria',when(dfuc['Categoria'] == 'liftback','SUV').when(dfuc['Categoria'] == 'pickup','Pickup').when(dfuc['Categoria'] == 'universal','SUV').otherwise(dfuc['Categoria']))

In [0]:
categoria_new = dfuc.dropDuplicates(['Categoria'])
categoria_new.select('Categoria').show()

+-----------+
|  Categoria|
+-----------+
|      Hatch|
|        SUV|
|      Coupê|
|      Sedan|
|     Pickup|
|   Limosine|
|        Van|
|    Minivan|
|Conversível|
+-----------+



In [0]:
dfuc = dfuc.withColumn('Valor (R$)', round(dfuc['Preço (US$)'] * 5.5))

In [0]:
dfuc.show(5)

+---------+--------+-----------+--------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+----------+
|Montadora|  Modelo|Transmissão|     Cor|Kilometragem| Ano|Combustível|Tem GNV|Motor|Categoria|Preço (US$)|Duração do Anúncio (dias)|GNV|Valor (R$)|
+---------+--------+-----------+--------+------------+----+-----------+-------+-----+---------+-----------+-------------------------+---+----------+
|   Subaru| Outback| Automático|   Prata|      190000|2010|   Gasolina|  False|  2.5|      SUV|    10900.0|                       16|Não|   59950.0|
|   Subaru| Outback| Automático|    Azul|      290000|2002|   Gasolina|  False|  3.0|      SUV|     5000.0|                       83|Não|   27500.0|
|   Subaru|Forester| Automático|Vermelho|      402000|2001|   Gasolina|  False|  2.5|      SUV|     2800.0|                      151|Não|   15400.0|
|   Subaru| Impreza|     Manual|    Azul|       10000|1999|   Gasolina|  False|  3.0|    Sedan|     9999.0

In [0]:
dfuc = dfuc.drop('Tem GNV').drop('Preço (US$)')

In [0]:
dfuc.show(5)

+---------+--------+-----------+--------+------------+----+-----------+-----+---------+-------------------------+---+----------+
|Montadora|  Modelo|Transmissão|     Cor|Kilometragem| Ano|Combustível|Motor|Categoria|Duração do Anúncio (dias)|GNV|Valor (R$)|
+---------+--------+-----------+--------+------------+----+-----------+-----+---------+-------------------------+---+----------+
|   Subaru| Outback| Automático|   Prata|      190000|2010|   Gasolina|  2.5|      SUV|                       16|Não|   59950.0|
|   Subaru| Outback| Automático|    Azul|      290000|2002|   Gasolina|  3.0|      SUV|                       83|Não|   27500.0|
|   Subaru|Forester| Automático|Vermelho|      402000|2001|   Gasolina|  2.5|      SUV|                      151|Não|   15400.0|
|   Subaru| Impreza|     Manual|    Azul|       10000|1999|   Gasolina|  3.0|    Sedan|                       86|Não|   54995.0|
|   Subaru|  Legacy| Automático|   Preto|      280000|2001|   Gasolina|  2.5|      SUV|          